In [ ]:
!pip install yfinance
import pandas as pd 
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore')
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq

     |████████████████████████████████| 6.3MB 7.4MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=4b0610891568bf3b07cf615b55eef494f8bda532b2f9f238e9908afc12542719
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
!pip install pytrends
from pytrends.request import TrendReq
pytrend = TrendReq()

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd '/content/gdrive/MyDrive/SASH/'

**Strategy 1**: After heavy trend on internet for 1week means market sentiment positive for upcoming week



In [ ]:
trade = yf.download('RELIANCE.NS', 
                   start='2020-05-31', 
                   end='2022-01-01',
                   progress=False,
                   interval='1wk')
#Pull trend of google search for particular stocks, example =  reliance
pytrend.build_payload(kw_list=['RELIANCE'],geo='IN',timeframe='today 12-m')
google = pytrend.interest_over_time()
google.reset_index(inplace=True)
# delta=pd.read_csv('weeklydelta.csv',sep=',',skiprows=1)
trade=trade[:-1]
trade.reset_index(drop=True,inplace=True)
trade['Trend']=google['RELIANCE']
trade=trade.drop(['Adj Close','Volume'],axis=1) # adding trend to weekly data
trade=trade.dropna().reset_index(drop=True)
from mlxtend.preprocessing import minmax_scaling #normalize
scaled_data1 = minmax_scaling(trade,columns=['Trend','Close'])
trade['Trend'] = (round(scaled_data1['Trend']*100).astype('int')).shift(1) # trend in percentage
# trade['2W Trend']= ((trade['Trend'] + trade['Trend'].shift(1))/2).shift(1) # last 2 week trend
# trade['Price'] = round(scaled_data1['Close']*100).astype('int') # price trend in percentage
trade.dropna(inplace=True)
trade.reset_index(drop=True,inplace=True)
df=trade[trade['Trend'] > 60] # greater than 60%
print('Points:',sum(df['Close'] - df['Open']))
print('Trades:',len(df))

Points: 153.849853515625
Trades: 7


**ISSUE:** Logic is not performing that well as people search on google mostly when price rise. Hence highly correlated data which is not providing any extra information.

_______________________________________________________________________________________________________________________________

STRATEGY 2: Using trend from youtube. If there is over trend then short else buy.

In [ ]:
def youtube_trend(stock,keyword): 
  trade = yf.download(stock, 
                    start='2020-05-31', 
                    end='2022-01-01',
                    progress=False,
                    interval='1wk')
  #Pull trend of google search for particular stocks, example =  reliance
  pytrend.build_payload(kw_list=[keyword],geo='IN',timeframe='today 12-m',gprop='youtube')
  google = pytrend.interest_over_time()
  google.reset_index(inplace=True)
  trade=trade[:-1]
  trade.reset_index(drop=True,inplace=True)
  trade['Trend']=google[keyword] # adding trend to weekly data
  trade=trade.drop(['Adj Close','Volume'],axis=1) 
  trade=trade.dropna().reset_index(drop=True)
  from mlxtend.preprocessing import minmax_scaling #normalize
  scaled_data1 = minmax_scaling(trade,columns=['Trend','Close'])
  trade['Trend'] = (round(scaled_data1['Trend']*100).astype('int')).shift(1) # trend in percentage
  trade.dropna(inplace=True)
  trade.reset_index(drop=True,inplace=True)
  buy=trade[trade['Trend'] < trade['Trend'].mean() ] # youtube trend lesser than its mean
  print('Buy Points:',sum(buy['Close'] - buy['Open']))
  print('Buy Trades:',len(buy))
  short=trade[trade['Trend'] > trade['Trend'].mean() ] # youtube trend greater than its mean
  print('Short Points:',sum(short['Open'] - short['Close']))
  print('Short Trades:',len(short))

In [ ]:
youtube_trend('APOLLO.NS','Apollo Tyre')

Buy Points: -4.849983215332031
Buy Trades: 33
Short Points: 34.100006103515625
Short Trades: 16


In [ ]:
!pip install GoogleNews
# !pip install newspaper3k

     |████████████████████████████████| 286kB 2.8MB/s 


In [ ]:
from GoogleNews import GoogleNews
# from newspaper import Article
import pandas as pd

In [ ]:
googlenews=GoogleNews()#period='1d',start='05/05/2021',end='30/05/2021')
googlenews.search('Fresher Data Scientist Role')
result=googlenews.result()
df=pd.DataFrame(result)
print(df.head())

                                               title  ...                                                img
0  DBA In Data Science - India's Only Applied Res...  ...  data:image/gif;base64,R0lGODlhAQABAIAAAP//////...
1  Expert Speaks: What Does A Career In Data Scie...  ...  data:image/gif;base64,R0lGODlhAQABAIAAAP//////...
2  What Employers are Expecting while Hiring Big ...  ...  data:image/gif;base64,R0lGODlhAQABAIAAAP//////...
3  3 Secret Mantras Nobody Knew to Woo Hiring Dat...  ...  data:image/gif;base64,R0lGODlhAQABAIAAAP//////...
4  Here is a list of government job openings in s...  ...  data:image/gif;base64,R0lGODlhAQABAIAAAP//////...

[5 rows x 7 columns]


In [ ]:
 
import os
for i in range(0,len(df)):
   print(df['title'][i])
   print(os.linesep)

DBA In Data Science - India's Only Applied Research Program


Expert Speaks: What Does A Career In Data Science Has In ...


What Employers are Expecting while Hiring Big Data ...


3 Secret Mantras Nobody Knew to Woo Hiring Data Science ...


Here is a list of government job openings in science and ...


Demand for techies nearly doubled in last 14 months: ISF


Tech Mahindra Announces Job and Educational Opportunities ...


India's demand for techies nearly doubled in last 14 months: ISF


Data Science is Losing its Value as a Top Tech Job. Here's Why!


Gregory keeps backpacks fresher, longer




In [ ]:
for i in range(0,5):
    googlenews.getpage(i)
    result=googlenews.result()
    df=pd.DataFrame(result)

In [ ]:
df1=df[df['date'].str.contains('hour|1 day')]

In [ ]:
for i in df1['title']:
  print(str(i))

Hank Ellis Insurance wins first-quarter men's golf league title
Metropolitan Control Centre Market Quality & Quantity ...
DC Power Supply: Where is Global Market Influencing from ...
Desiccated Coconut Market: Opportunities And Strategies size ...
Global Interactive Classroom Projectors Market Status, Future ...
Niger Delta group warns against implementing proposed ...
Wellhead Equipment Market (Covid-19 Analysis) SWOT ...
Digital Switch-Mode Power Supplies Market 2021 Analysis ...
Impact of Covid-19 on Oilfield Equipment Market 2020-2028 ...
Hank Ellis Insurance wins first-quarter men's golf league title
Metropolitan Control Centre Market Quality & Quantity ...
DC Power Supply: Where is Global Market Influencing from ...
Desiccated Coconut Market: Opportunities And Strategies size ...
Global Interactive Classroom Projectors Market Status, Future ...
Niger Delta group warns against implementing proposed ...
Wellhead Equipment Market (Covid-19 Analysis) SWOT ...
Digital Switch-Mode Powe

**Google client for latest headline news**

In [ ]:
# !pip install gnewsclient
from gnewsclient import gnewsclient
client = gnewsclient.NewsClient(language='english', 
                                location='india', 
                                topic='Technology', 
                                max_results=10)
news_list = client.get_news()
# for i in news_list:
#   if 'Reliance' in i['title']:
#     print(i['title'])

In [ ]:
news_list

[{'link': 'https://news.google.com/__i/rss/rd/articles/CBMiTGh0dHBzOi8vd3d3LmdzbWFyZW5hLmNvbS9vbmVwbHVzX25vcmRfY2VfNWdfZnVsbF9zcGVjc19sZWFrZWQtbmV3cy00OTQzMi5waHDSAUlodHRwczovL20uZ3NtYXJlbmEuY29tL29uZXBsdXNfbm9yZF9jZV81Z19mdWxsX3NwZWNzX2xlYWtlZC1hbXAtNDk0MzIucGhw?oc=5',
  'media': None,
  'title': 'OnePlus Nord CE 5G full specs leaked - GSMArena.com news - GSMArena.com'},
 {'link': 'https://news.google.com/__i/rss/rd/articles/CBMigQFodHRwczovL2dhZGdldHMubmR0di5jb20vbW9iaWxlcy9uZXdzL3JlYWxtZS01Zy1nbG9iYWwtc3VtbWl0LWRhdGUtdGltZS10b2RheS0yLTMwLXBtLWhvdy10by13YXRjaC1saXZlc3RyZWFtLWd0LWV4cGVjdGVkLTI0NTQ4MjjSAZYBaHR0cHM6Ly9nYWRnZXRzLm5kdHYuY29tL21vYmlsZXMvbmV3cy9yZWFsbWUtNWctZ2xvYmFsLXN1bW1pdC1kYXRlLXRpbWUtdG9kYXktMi0zMC1wbS1ob3ctdG8td2F0Y2gtbGl2ZXN0cmVhbS1ndC1leHBlY3RlZC0yNDU0ODI4P2FtcD0xJmFrYW1haS1ydW09b2Zm?oc=5',
  'media': None,
  'title': 'Realme 5G Global Summit Today, Realme GT 5G Launch Expected: How to Watch Livestream - Gadgets 360'},
 {'link': 'https://news.google.com/__i/rss/rd/a